### Running common notebook to get access to variables

In [0]:
%run "/Workspace/Users/khanhminh12723_gmail.com#ext#@khanhminh12723gmail.onmicrosoft.com/04. Common notebook"

# To re-use common functions and variables


# Re-Using functions
- We can re-use 2 functions here
  - Removing Duplicates
  - Removing NULLs

In [0]:
print(checkpoint)

abfss://checkpoints@databricksdevstrg.dfs.core.windows.net/


# Defining all common variables

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")

## Defining common functions

### 01 -- Removing duplicates

### 01 -- Handling NULLs

### Reading from bronze raw_Roads

In [0]:
def read_BronzeRoadsTable(environment):
    print('Reading the Bronze Table raw_roads Data : ',end='')
    df_bronzeRoads = (spark.readStream
                    .table(f"`{environment}`.`bronze`.raw_roads")
                    )
    print(f'Reading {environment}.bronze.raw_roads Success!')
    print("**********************************")
    return df_bronzeRoads

In [0]:
df_roads = read_BronzeRoadsTable(env)

Reading the Bronze Table raw_roads Data : Reading databricks_dev_wp.bronze.raw_roads Success!
**********************************


### Creating road_category_name column

In [0]:
def road_Category(df):
    print('Creating Road Category Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Cat = df.withColumn("Road_Category_Name",
                  when(col('Road_Category') == 'TA', 'Class A Trunk Road')
                  .when(col('Road_Category') == 'TM', 'Class A Trunk Motor')
                   .when(col('Road_Category') == 'PA','Class A Principal road')
                    .when(col('Road_Category') == 'PM','Class A Principal Motorway')
                    .when(col('Road_Category') == 'M','Class B road')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Cat

### Creating road_type column

In [0]:
def road_Type(df):
    print('Creating Road Type Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Type = df.withColumn("Road_Type",
                  when(col('Road_Category_Name').like('%Class A%'),'Major')
                  .when(col('Road_Category_Name').like('%Class B%'),'Minor')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Type

### Writing data to silver_roads in Silver schema

In [0]:
def write_Roads_SilverTable(StreamingDF,environment):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/SilverRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}`.`silver`.`silver_roads`"))
    
    write_StreamSilver_R.awaitTermination()
    print(f'Writing `{environment}_catalog`.`silver`.`silver_roads` Success!')

### Calling functions

In [0]:
df_noDups = remove_Dups(df_roads)

AllColumns = df_noDups.schema.names
df_clean = handle_NULLs(df_noDups,AllColumns)

## Creating Road_Category_name 
df_roadCat = road_Category(df_clean)

## Creating Road_Type column
df_type = road_Type(df_roadCat)

## Writing data to silver_roads table

write_Roads_SilverTable(df_type,env)

Removing Duplicate values: Success!
Replacing NULLs of Strings DataType with "Unknown": Success!
Replacing NULLs of Numeric DataType with "0":  Success!
Creating Road Category Name Column: Success!! 
***********************
Creating Road Type Name Column: Success!! 
***********************
Writing the silver_roads Data : Writing `databricks_dev_wp_catalog`.`silver`.`silver_roads` Success!


In [0]:
%sql

SELECT * FROM `databricks_dev_wp`.`silver`.`silver_roads`

Road_ID,Road_Category_Id,Road_Category,Region_ID,Region_Name,Total_Link_Length_Km,Total_Link_Length_Miles,All_Motor_Vehicles,Road_Category_Name,Road_Type
23,3,TA,6,London,339.479,210.94,3.913260559E9,Class A Trunk Road,Major
39,5,M,9,South East,40851.1,25383.7,1.5597621429E10,Class B road,Minor
31,2,PM,8,Yorkshire and the Humber,5.612,3.49,7.880772228E7,Class A Principal Motorway,Major
13,1,TM,4,Wales,130.291,80.96,1.306589845E9,Class A Trunk Motor,Major
65,5,M,4,Wales,28370.8,17628.8,5.374885013E9,Class B road,Minor
67,2,PM,5,North West,38.642,24.01,3.055495587E8,Class A Principal Motorway,Major
12,5,M,3,Scotland,47971.6,29808.17,7.656438848E9,Class B road,Minor
52,4,PA,1,South West,3882.154,2412.26,8.028746829E9,Class A Principal road,Major
29,5,M,7,East of England,34610.8,21506.15,1.1083436184E10,Class B road,Minor
53,5,M,1,South West,43659.6,27128.82,9.272591264E9,Class B road,Minor
